In [1]:
! pip install ..
import numpy as np
import math
from scipy.stats import poisson
from dismal import matrices, likelihood2
from scipy import linalg
from dismal.likelihood2 import *

Processing /Users/s2341012/Dropbox/DISMaL_chapter/DISMaL
  Preparing metadata (setup.py) ... done
  Created wheel for dismal: filename=dismal-0.1-py3-none-any.whl size=13862 sha256=d3f4b9126d6270f054e1def4873fa56ddf4aa319a4706a8be6e4bb0cb723d30e
  Stored in directory: /private/var/folders/qk/10crn8pd5_d038f0tw_7z2800000gp/T/pip-ephem-wheel-cache-qi4tsk3x/wheels/10/c9/59/9e87bfb89308645a07e4be9051b0645d49425b51da0dd9ddc7
Successfully built dismal
  Attempting uninstall: dismal
    Found existing installation: dismal 0.1
    Uninstalling dismal-0.1:
      Successfully uninstalled dismal-0.1


In [2]:
q1 = matrices.GeneratorMatrix(c1=1, c2=1, m1_prime=0, m2_prime=0, matrix_type="Q1")
q2 = matrices.GeneratorMatrix(b=1, m1=0, m2=0, matrix_type="Q2")
q3 = matrices.GeneratorMatrix(a=1, matrix_type="Q3")

In [3]:
likelihood2.likelihood_matrix(q1, q2, q3, 1, 1, [[0], [0], [0]])

[array([-0.48168436]), array([-0.48168436]), array([0.06766764])]

In [5]:
S = [[0], [0], [0]]
i = 0
t1 = 1
v = 1
g, eigvals_q1 = q1.eigen()
c, eigvals_q2 = q2.eigen()
ginv = linalg.inv(g)
cinv = linalg.inv(c)
alpha = -eigvals_q1[0:3]
beta = -eigvals_q2[0:3]
gamma = 1/q3.pop_size1 #1/a

gg = ginv @ np.diag(g[:,3])
cc = cinv @ np.diag(c[:,3])
pij1 = p_matrix(g, ginv, eigvals_q1, t1)
pij2 = p_matrix(c, cinv, eigvals_q2, v)

gg[i, 0:3] @ alpha_matrix(alpha, S[i], t1) + pij1[i, 0:3] @ cc[0:3, 0:3] @ beta_matrix(beta, S[i], t1=t1, t0=(v+t1)) + (1 - (pij1@pij2)[i,3]) * gamma_matrix(gamma,S[i],t1+v)

array([-0.48168436])

In [14]:
pij2

array([[0.36787944, 0.        , 0.        , 0.63212056],
       [0.        , 0.36787944, 0.        , 0.63212056],
       [0.        , 0.        , 1.        , 0.        ],
       [0.        , 0.        , 0.        , 1.        ]])

In [5]:
gg[i, 0:3] @ alpha_matrix(alpha, S[i], t1)

array([-0.43233236])

In [16]:
gg[0,0:3]

array([ 1., -1.,  0.])

In [6]:
alpha_matrix(alpha, S[0], t1)

array([[-0.        ],
       [ 0.43233236],
       [ 0.43233236]])

In [7]:
def alpha_matrix(alpha, s_vals, t1, rel_mu=1):
    alpha, s_vals = np.array(alpha), np.array(s_vals)
    alphas = []
    for s in s_vals:
        alphas.append((alpha/(alpha+rel_mu)) * ((rel_mu/(alpha+rel_mu))**s) * (1-poisson.cdf(s, t1*(alpha+rel_mu))))
    return np.transpose(np.array(alphas))

In [8]:
rel_mu=1
s = 0
(alpha/(alpha+rel_mu)) * ((rel_mu/(alpha+rel_mu))**s) * (1-poisson.cdf(s, t1*(alpha+rel_mu)))

array([-0.        ,  0.43233236,  0.43233236])

In [9]:
s=0
t0=2
t1=1
beta = np.array([1,1,0])
(beta/(beta+rel_mu)) * ((rel_mu/(beta+rel_mu))**s) * np.exp(beta*t1) * (poisson.cdf(s, t1*(beta+rel_mu) - poisson.cdf(s, t0*(beta+rel_mu))))

array([0.18733974, 0.18733974, 0.        ])

In [10]:
[(beta/(beta+rel_mu)), ((rel_mu/(beta+rel_mu))**s), np.exp(beta*t1)]

[array([0.5, 0.5, 0. ]),
 array([1., 1., 1.]),
 array([2.71828183, 2.71828183, 1.        ])]

In [11]:
[poisson.cdf(s, t1*(beta+rel_mu)), poisson.cdf(s, t0*(beta+rel_mu))]

[array([0.13533528, 0.13533528, 0.36787944]),
 array([0.01831564, 0.01831564, 0.13533528])]

In [12]:
poisson.cdf(s, (t1*(beta+rel_mu))) # OK

array([0.13533528, 0.13533528, 0.36787944])

In [13]:
poisson.cdf(s, t0*(beta+rel_mu))

array([0.01831564, 0.01831564, 0.13533528])

In [14]:
#(beta/(beta+R1))*(R1/(beta+R1))^X1*exp(beta*T1)*(ppois(X1,T1*(beta+R1))-ppois(X1,T0*(beta+R1)))
(beta/(beta+rel_mu)) * ((rel_mu/(beta+rel_mu))**s) * np.exp(beta*t1) * (poisson.cdf(s, t1*(beta+rel_mu)) - poisson.cdf(s, t0*(beta+rel_mu)))

array([0.15904619, 0.15904619, 0.        ])